In [1]:
import wave
import os
import librosa 

In [3]:
Path_to_train = "C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav"
subfolders = os.listdir(Path_to_train)
data = []
for s in subfolders:
    files = os.listdir(Path_to_train + "/" +s)
    data.extend([Path_to_train + "/" + s+ "/" + f for f in files])
data[:10]

['C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part10.wav',
 'C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part100.wav',
 'C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part101.wav',
 'C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part102.wav',
 'C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part103.wav',
 'C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SWH-05-20101106/SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part104.wav',
 'C:/Users/DESMOND/NLP/ALFFA_PUBLIC/ASR/SWAHILI/data/train/wav/SW

In [8]:
#read text from every transcription audio
def read_text(text_path):
    text = []
    with open(text_path) as fp:
        line = fp.readline()
        while line:
        # TODO: fix spaces in in amharic text
            text.append(line)
            line = fp.readline()
    return text

In [9]:
#extract the transcription and the label 
label=[]
transcriptions = []
for t in text:
    sp = t.split("\t")
    sp = sp.strip("\n")
    if len(sp) > 1:
        label.append(sp[0])
        transcriptions.append(sp[1])
transcriptions[:10]

NameError: name 'text' is not defined

In [10]:
#get audio path , every path must corespond to transcription , get the transprion in the doc and append to audio path 
audio_path=[0]*len(transcriptions)
for d in data:
    _d = d.strip(".wav")
    sp = _d.split("/")[2]
    index = label.index(sp)
    audio_path[index] = d

ValueError: 'DESMOND' is not in list

In [11]:
#calculate duration 
duration_of_recordings=[]
for d in audio_path:
    audio, fs = librosa.load(d, sr=None)
    duration_of_recordings.append(float(len(audio)/fs))

In [12]:
import pandas as pd 
data=pd.DataFrame({'key': audio_path,'text': transcriptions, 'duration':duration_of_recordings})

In [13]:
data

Empty DataFrame
Columns: [key, text, duration]
Index: []

In [14]:
#before using this function kindly change your file paths for it to work


def make_stereo(audio_path):
    #this function converts mono audio channels into stereo channels 
#     logging.info(" ============ Conerting audio sample from mono to stereo ================= ")
    print("======= Mono to stereo audio conversion")
    ifile = wave.open(audio_path)
    #log the info on adio files
#     logging.info(ifile.getparams())
    print (ifile.getparams())
    # (1, 2, 44100, 2013900, 'NONE', 'not compressed')
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = ifile.getparams()
    assert (comptype == 'NONE')  # Compressed not supported yet
    array_type = {1:'B', 2: 'h', 4: 'l'}[sampwidth]
    print(" ======= Calculting left channel type =====")
    left_channel = array.array(array_type, ifile.readframes(nframes))[::nchannels]
    ifile.close()

    #convert the number of channels to 2
    print("====== converting channels ======= ")
    stereo = 2 * left_channel
    stereo[0::2] = stereo[1::2] = left_channel
    #overwrite the wav file making it a stereo file
    print("====== overwriting wav file ======= ")
    ofile = wave.open(audio_path, 'w')
    ofile.setparams((2, sampwidth, framerate, nframes, comptype, compname))
    ofile.writeframes(stereo.tostring())
    ofile.close()


In [15]:
data['key']=data['key'].apply(lambda x: make_stereo(x))